# 3. 連立一次方程式
### 概要

- 連立一次方程式の解法は二つに分類できる
- 消去法（LU分解）
- 反復法

### key-words

- 吐き出し法
- ガウスの消去法
- LU分解
- 逆行列
- ガウス・ザイデル法
- SOR法


In [1]:
!python -V

Python 3.7.4


Import

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu

### 注意

- numpyを用いた行列表現はarrayを使用する
- `*`演算をするとアダマール積になるが`@`を用いることで通常の行列の積の実行結果を得ることができる
- Scipy.orgにおける`numpy.matrix` documentにも`Note　It is no longer recommended to use this class, even for linear algebra. Instead use regular arrays. The class may be removed in the future.`とのこと

### 吐き出し法

次のルールを使って、未知数を一つずつ消していく手法

$$
\begin{aligned}
2x + 2y + 6z &= 24 \\
3x + 5y + 13z &= 52\\
5x + 8y + 24z &= 93
\end{aligned}
$$
を考えるとき、次の二元配列（行列）に変換する

$$
\left(
\begin{array}{cccc}
2 & 2 & 6 & 24 \\
3 & 5 & 13 & 52\\
5 & 8& 24 & 93
\end{array}
\right)
$$

- ある行をその行に対応するピボット（対角成分）で割る
- ある行から他の行の定数倍を引く
- ある行と他の行を入れ替える

#### Pseudocode

```
1. Start

2. Input the Augmented Coefficients Matrix (A):
	
	For i = 1 to n
		
		For j = 1 to n+1
			
			Read Ai,j
		
		Next j
	
	Next i

3. Apply Gauss Jordan Elimination on Matrix A:
	
	For i = 1 to n
		
		If Ai,i = 0
			
			Print "Mathematical Error!"
			Stop
		
		End If
		
		For j = 1 to n
			
			If i ≠ j 
				
				Ratio = Aj,i/Ai,i
				
				For k = 1 to n+1
				
					Aj,k = Aj,k - Ratio * Ai,k
			
				Next k
				
			End If
			
		Next j
	Next i

4. Obtaining Solution:
	
	For i = 1 to n 
		Xi = Ai,n+1/Ai,i
	Next i

5. Display Solution:
	
	For i = 1 to n
		
		Print Xi
	
	Next i

6. Stop


---------------
Note: All array indexes are assumed to start from 1.
```


### [例題1]

$$
\begin{aligned}
2x + 2y + 6z &= 24 \\
3x + 5y + 13z &= 52\\
5x + 8y + 24z &= 93
\end{aligned}
$$

を解け


In [3]:
def gauss_jordan(A, output_array = None):
    n, m = A.shape
    for i in range(n):
        if A[i, i] == 0:
            raise ValueError('{}-th pivot is zero'.format(i))
        
        A[i, :] = A[i, :]/A[i,i]
        
        for j in range(n):
            if i != j:
                A[j, :] = A[j, :] - A[j,i] * A[i, :]
    if output_array:
        return A
    return A[:, -1]
                

In [4]:
X = np.array(([2, 2, 6, 24] 
             ,[3, 5, 13, 52]
             ,[5, 8, 24, 93]))
gauss_jordan(X, output_array = True)

array([[1, 0, 0, 1],
       [0, 1, 0, 2],
       [0, 0, 1, 3]])

In [5]:
%%timeit
gauss_jordan(X)

39.3 µs ± 6.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
A, b = X[:, :-1], X[:, -1]

In [7]:
%%timeit
np.linalg.solve(A, b)

12.1 µs ± 973 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


- 自作コードは`for loop`をネストで使用しているため、計算時間がとんでもなく遅いことに留意

### ガウスの消去法: Gauss Elimination Method

- 計算時間が短いので広く利用されている
- ガウスの消去法は前進消去（文字を1つずつ消していく操作）と後退代入（1成分ずつ答えを求めていく操作）からなる

#### 前進消去法のゴール

$$
\left(
\begin{array}{cccc}
u_{11} & u_{12} & u_{13} & b_1 \\
0 & u_{22} & u_{23} & b_2\\
0 & 0 & u_{33} & b_3
\end{array}
\right)
$$

- 後退代入は後ろの式から順番にたどっていき，一つずつ変数の値を求めていくだけ

#### ガウス消去法のアルゴリズム

```
[INPUT]
    A = (a[i,j]): 係数行列(n次の正方行列)
    b = (b[i]): 定数ベクトル
    
[OUTPUT]
    x: 解ベクトル
    
[PROCESS]
    (前進消去法)
        各列k = 1, ..., n-1について次の処理を反復する
        (k列の吐き出し)
            各行 i = k+1, ..., nについて次の処理を反復する
            (a[i,k]の掃き出し)
                pivot = a[k,k]
                ratio = a[i,k]/pivot
                for (j == k+1, j<= n, j++):
                    a[i,j] = a[i,j] - a[k, j] * ratio
                b[i] = b[i] - b[k]*ratio
     
     (後退代入)
         for (k = n, k >= 1, k--):
             x[k] = b[k]
             for (j == k+1, j<= n, j++):
                 x[k] = x[k] - a[k,j]*x[j]
             
             x[k] = x[k]/a[k,k]
```



#### Pseudocode for Gauss Elimination Method

<img src = "https://github.com/RyoNakagami/omorikaizuka/blob/master/pseudocode/gauss_elimination.jpg?raw=true">

- Psuedocodeの`maxEl`はpivot == 0に起因する計算エラーを防ぐために実施している
- 絶対値が大きいpivotを探し出し、スイッチしている（ピボット選択 or 部分ピボット選択という）
- pivotの絶対値はスケールに依存するので、通常は各行の絶対値最大の要素でその行の要素を割るというスケーリング前処理をする

### [例題2]
ガウスの消去法を用いて、次の連立一次方程式を解け

$$
\begin{aligned}
2x + 2y + 6z &= 24 \\
3x + 5y + 13z &= 52\\
5x + 8y + 24z &= 93
\end{aligned}
$$

In [8]:
def gauss_elimination(X):
    A = X.copy()
    n, m = A.shape
    for i in range(0, n):
        # Search for maximum 
        maxEl = abs(A[i][i])
        maxRow = i
        for k in range(i+1, n):
            if abs(A[k][i]) > maxEl:
                maxEl = abs(A[k][i])
                maxRow = k

        if i != maxRow:
            tmp = A[i, :].copy()
            A[i, :] = A[maxRow, :]
            A[maxRow, :] = tmp


        # 対角成分以下をゼロ
        for k in range(i+1, n):
            c = -A[k][i]/A[i][i]
            for j in range(i, n+1):
                if i == j:
                    A[k][j] = 0
                else:
                    A[k][j] += c * A[i][j]

    # Solve equation Ax=b
    x = [0 for i in range(n)]
    for i in range(n-1, -1, -1):
        x[i] = A[i][n]/A[i][i]
        for k in range(i-1, -1, -1):
            A[k][n] -= A[k][i] * x[i]
    return x

In [9]:
gauss_elimination(X)

[1.0, 2.0, 3.0]

In [10]:
%%timeit
gauss_elimination(X)

96.9 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### LU分解：LU Decomposition

LU stands for ‘Lower Upper’, and so an LU decomposition of a matrix A is a decomposition so that

$$
A = LU
$$

- 基本的にはGaussian elimination
- 行列に特異性がない限り(経験的に)安定に解ける
- A が正則 ⇒ A が適当な列変換により LU 分解可能
- 一般の行列で次数が多くない場合には迷わず選択
- 計算量はO(n^3)

#### 処理方針

1. 行列Aのほかに、単位行列Iを準備しこの行列をMとする
2. AをGaussian eliminationすると同時に、それぞれの乗数 $m_{ik}$ を行列Mの(i, k)成分として記憶する
3. アルゴリズムが終了したときのMをL, AをUとする


In [11]:
import scipy.linalg as la

A = np.array([[1,3,4],[2,1,3],[4,1,2]])

print(A)

P, L, U = la.lu(A)
print(L@U)
print(P@L@U)

[[1 3 4]
 [2 1 3]
 [4 1 2]]
[[4. 1. 2.]
 [1. 3. 4.]
 [2. 1. 3.]]
[[1. 3. 4.]
 [2. 1. 3.]
 [4. 1. 2.]]


In [12]:
print(P)

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [13]:
print(L)

[[1.         0.         0.        ]
 [0.25       1.         0.        ]
 [0.5        0.18181818 1.        ]]


In [14]:
print(U)

[[4.         1.         2.        ]
 [0.         2.75       3.5       ]
 [0.         0.         1.36363636]]


### LU DecompositionのPsuedocode

```
[OUTPUT]
    P : Permutation matrix
    L : Lower triangular
    U : Upper triangular

[REMARK]
    P @ L @ U = A
    P @ A = L @ U

function lu_decompose(A: matrix)
    n = nrow(A)
    P = indentity matrix, which is N times N
    //row swap
    for (i = 1; i <= n, i++) do
         maxEl = |A[i,i]|
         maxRow = i
      
         for (k = i + 1; k <= n; k++) dp
             if |A[k, i]| > maxEl then
                 maxEl = A[k, i]
                 maxRow = k
             end if
         end for
         
         A[maxRow, :], A[i, :] = A[i, :], A[maxRow, :]
         P[maxRow , i], P[i, i]  = 1, 0
    end for
    
    L = I, which is N times N identity matrix
    
    for (i = 1; i <= n, i++) do
        for (k = i + 1; k <= n; k++) do
            m = A[k, i]/A[i, i]
            A[k, :] = A[k, :] - A[i, :] * m
            L[k, i] = m
        end for
    end for
    
    return P, L, A
```

In [15]:
def lu_decomposition(X):
    A = X.copy()
    n = A.shape[0]
    P = np.identity(n)
    L = np.identity(n)
    
    for i in range(n):
        maxEl = abs(A[i, i])
        maxRow = i
        
        for k in range(i+1, n):
            if abs(A[k, i]) > maxEl:
                maxEl = abs(A[k, i])
                maxRow = k
                
        if i != maxRow:
            P[i, i], P[i, maxRow], P[maxRow, i], P[maxRow, maxRow] = 0, 1, 1, 0
            tmp = A[i, :].copy()
            A[i, :] = A[maxRow, :]
            A[maxRow, :] = tmp
    for i in range(n):
        for k in range(i + 1, n):
            m = A[k, i]/A[i, i]
            A[k, :] = A[k, :] - A[i, :] * m
            L[k, i] = m
        
    
    return P, L, A
        

### [例題3]

$$
A = \left(\begin{array}{ccc}
2&2&6\\
3&5&13\\
5&8&24
\end{array}\right)
$$
をLU分解せよ

In [16]:
A = np.array(([2., 2., 6.]
             ,[3., 5., 13.]
             ,[5., 8., 24.]))

P, L, U = lu_decomposition(A)
print(L)
print(U)
print(P)

[[ 1.   0.   0. ]
 [ 0.6  1.   0. ]
 [ 0.4 -6.   1. ]]
[[  5.    8.   24. ]
 [  0.    0.2  -1.4]
 [  0.    0.  -12. ]]
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [17]:
print(P @ L @ U)

[[ 2.  2.  6.]
 [ 3.  5. 13.]
 [ 5.  8. 24.]]


### LU分解アルゴリズムの正当性
#### 定理

入力Aに対するLU分解のアルゴリズムの出力がL, Uならば$LU = A$

#### [証明]

任意の計算状態$<M_t, A_t>$において$M_tA_t = A$であることを帰納法を用いて示す。

$t = 0$のときは $M_0A_0 = IA = A$より自明。

次に、帰納法の仮定
$$
M_t A_t = A
$$

のもとで

$$
M_{t+1} A_{t+1} = A
$$

を示す。

第$t+1$ stepで掃き出すのが$A_t$の(i, k)成分であるので、アルゴリズムより$ i > k$, このときの乗数を$m_{ik}$とする。ここで単位行列の(i, k)成分をそれぞれ$m_{i,k}, -m_{i,k}$としたものをそれぞれS, Tとする。

$$
S = \left(\begin{array}{cccccc}
1 & 0 & ...& ...& ...& 0\\
0 & ... & ...& ...& ...& 0\\
0 & ... & 1 & ... & ...& ...\\
...& ... & ...& ...& ...& ...\\
0 & ... & m_{ik}& ... & ...& ...\\
0 & ... & ...& 0& ...& 1
\end{array}\right)
$$

$$
T = \left(\begin{array}{cccccc}
1 & 0 & ...& ...& ...& 0\\
0 & ... & ...& ...& ...& 0\\
0 & ... & 1 & ... & ...& ...\\
...& ... & ...& ...& ...& ...\\
0 & ... & -m_{ik}& ... & ...& ...\\
0 & ... & ...& 0& ...& 1
\end{array}\right)
$$

このとき以下の性質を用いる

1. $ST = I$
2. Sを任意の行列Mの右から掛けると、Mの第i列の$m_{ik}$倍をMの第k列に加える操作に相当する

よって
$$
\begin{aligned}
M_{t + 1} &= M_t S\\
A_{t + 1} & = T A_t
\end{aligned}
$$

Then,

$$
M_{t + 1}A_{t + 1} = (M_t S)(T A_t) = M_t A_t = A
$$

よって任意のtについて$M_t A_t = A$. Then $LU = M_{T}A_{T} = A$ where T は最終ステップ。

### LUx = bを解くアルゴリズム

```
[INPUT]
    L, U: LU分解
    B: 定数ベクトル
    
[OUTPUT]
    x: solution
    
[PROCESS]
    n = nrows(L)
    (前進代入)
        y = b
        for (i = 1; i <= n; i++) do
            for (j = 1; j <= n; j++) do
                y[i] = y[i] - L[i, j] * y[j]
            end for
        end for
    
    {後退代入}
        x = y
        for (i = n; i >= 1; i--) do
            for (j = i + 1; i <= n; i++) do
                x[i] = x[i] - U[i, j] * x[j]
            end for
            x[i] = x[i]/U[i, i]
        end for
        
```


### [例題4] LU分解を用いて連立一次方程式を解く

$$
\begin{aligned}
2x + 2y + 6z &= 24 \\
3x + 5y + 13z &= 52\\
5x + 8y + 24z &= 93
\end{aligned}
$$

In [18]:
A = np.array(([2., 2., 6.] 
             ,[3., 5., 13.]
             ,[5., 8., 24.]))
b = np.array([24., 52., 93.])

In [19]:
P, L, U = lu(A)
L

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.4       ,  1.        ,  0.        ],
       [ 0.6       , -0.16666667,  1.        ]])

In [20]:
def lu_linear_slove(P, L, U, b):
    #前進代入
    y = P @ b.copy()
    n = len(b)
    for i in range(n):
        for j in range(i):
            y[i] = y[i] - L[i, j] * y[j]
            
    #後退代入
    for i in range(n-1, -1, -1):
        for j in range(i + 1, n):
            y[i] = y[i] - U[i, j] * y[j]
        
        y[i] = y[i]/U[i, i]
    return y

In [21]:
P, L, U = lu_decomposition(A)

In [22]:
print(L)

[[ 1.   0.   0. ]
 [ 0.6  1.   0. ]
 [ 0.4 -6.   1. ]]


In [23]:
lu_linear_slove(P, L, U, b)

array([1., 2., 3.])

### [例題5] 連立方程式の利用

A さんは午前 10 時に家を出発し，自転車に乗って時速 12km で走り， 午前 11 時 30 分に目的地に着く予定であった.ところが，途中で自転車が故障したので，そこからは時速 4km で歩いた.そのため，目的地に着いたのは出発してから 2時間後の正午であった.家から自転車が故障した地点までの道のりを求めなさい.

#### 解答方針
$$
\begin{aligned}
12x + 4y & = 18\\ 
x + y & = 2
\end{aligned}
$$

In [24]:
A = np.array([[12., 4.], [1., 1.]])
b = np.array([[12.0 * 1.5], [2.0]])

x = gauss_elimination(np.hstack([A, b]))
print(A @ x, x)

[18.  2.] [1.25, 0.7499999999999999]


### 逆行列の導出

In [25]:
A = np.array([[2. , -1. , 0], [-1., 2., -1.], [0., -1., 2]])
I = np.identity(3) 
inverse = gauss_jordan(np.hstack([A, I]), output_array = True)[:, 3:]

In [26]:
inverse

array([[0.75, 0.5 , 0.25],
       [0.5 , 1.  , 0.5 ],
       [0.25, 0.5 , 0.75]])

In [27]:
np.linalg.inv(A)

array([[0.75, 0.5 , 0.25],
       [0.5 , 1.  , 0.5 ],
       [0.25, 0.5 , 0.75]])

数値計算的な立場からは、逆行列は追放されるべきで、それに代わってLU分解を活躍させるべきとされる。

## 逆行列さようなら

以下の線形方程式を解く場合

$$
Ax = b
$$

逆行列を用いて

$$
x = A^{-1}b
$$

と紹介されるが、計算回数が多くなってしまい、LU分解を用いて解くことを推奨される。

In [28]:
def matrix_inv(X):
    n = len(X)
    A = X.copy()
    for k in range(n):
        w = 1/A[k, k] # n回の除算
        A[k, k] = w
        for j in range(n):
            if j != k:
                A[k, j] = w * A[k, j] #n(n-1)回の乗算
        for i in range(n):
            if i != k:
                for j in range(n):
                    if j != k:
                        # n(n-1)^2回の減算と乗算
                        A[i, j] = A[i, j] - A[i, k] * A[k, j]
        for i in range(n):
            if i != k:
                # n(n-1)回の乗算
                A[i, k] = - w * A[i, k]
    return A

逆行列を計算すると乗算が$n^3$, 減算が$n(n - 1)^2$回。さらに、$A^{-1}$とbの積の計算に乗算$n^2$, 加算$n(n-1)$が必要になる。

In [29]:
matrix_inv(A)

array([[0.75, 0.5 , 0.25],
       [0.5 , 1.  , 0.5 ],
       [0.25, 0.5 , 0.75]])

### LU分解の計算回数

```
[INPUT]
    n times n matrix

[OUTPUT]
    対角線喪服めちゃ右上算かく部分にU、左下部分にLの非対角非ゼロ要素が格納された行列を返す

for k = 1 to n-1 do
    w = 1/a[k,k] # n-1回
    
    for i = k+1 to n do
         a[i,k] = w * a[i,k]　# \sum_{i = i}^{n-1} i = n(n-1)/2
         
         for j = k+1 to n do
             a[i,j] = a[i,j] - a[i,k]*a[k,j]  # \sum^{n-1} i^2\
```

乗除算は

$$
\begin{aligned}
(n-1) + \sum_{i = 0}^{n-1} (n - i) +  \sum_{i = 0}^{n-1} (n - i)^2 & = (n - 1) + n(n-1)/2 + \sum^{n-1} i^2\\
& = (n - 1)\frac{3n + 6 + 2n^2 - n}{6}\\
& = (n - 1)(n^2 + n + 3)/6
\end{aligned}
$$ 

減算は同様に

$$
n(n - 1)(2n-1)/6
$$

In [30]:
def lu_simple(X):
    A = X.copy()
    n = len(A)
    for k in range(n - 1):
        w = 1/A[k, k]
        
        for i in range(k+1, n):
            A[i,k] = w * A[i,k]
            for j in range(k+1, n):
                A[i,j] = A[i,j] - A[i,k]*A[k,j]
    
    return A

In [31]:
lu_simple(A)

array([[ 2.        , -1.        ,  0.        ],
       [-0.5       ,  1.5       , -1.        ],
       [ 0.        , -0.66666667,  1.33333333]])

LU分解の演算回数と逆行列の演算回数を比較するとLU分解の方がおよそ1/3程度になることがわかる。また線形方程式の解を得る演算回数も逆行列をbに掛け合わせた時と同じ演算回数になることが知られている。よってLU分解を用いる方が良いとされる。

## 行列式とLU

$$
\text{det}(A) = \text{det}(L)\times \text{det}(U)=\text{det}(U)
$$

- Lは対角成分全部1なので行列式は1
= $\text{det}(U)$は対角成分の積で計算可能

## 反復法

次のような疎行列を考える

$$
A = \begin{equation}
\begin{bmatrix}
A_{-n,0}^{-n}  & A_{-n+1,-1}^{-n}  & 0 & \cdots & \cdots & \cdots & \cdots & 0 \\
A_{-n,1}^{-n+1}  & A_{-n+1,0}^{-n+1}  & A_{-n+2,-1}^{-n+1}  & \ddots & && & \vdots \\
0 & A_{-n+1,1}^{-n+2}  & A_{-n+2,0}^{-n+2} & A_{-n+3,-1}^{-n+2}  & \ddots & &  & \vdots \\
\vdots & \ddots & \ddots & \ddots & \ddots & \ddots &  & \vdots \\
\vdots & & \ddots & \ddots & \ddots & \ddots & \ddots& \vdots\\
\vdots  & & & \ddots & A_{n-3,1}^{n-2}  & A_{n-2,0}^{n-2}  &  A_{n-1,-1}^{n-2}  & 0\\
\vdots  & && & \ddots & A_{n-2,1}^{n-1}  & A_{n-1,0}^{n-1}  &  A_{n,-1}^{n-1}\\
0 & \cdots &  \cdots & \cdots & \cdots & 0 & A_{n-1,1}^{n}  & A_{n,0}^{n}  \\
\end{bmatrix}
\end{equation}
$$

non-zero成分が対角線付近にだけ存在する特徴的な形をしているこのような行列は帯行列と呼ばれている。隣接した未知数の間だけで直接の相互作用があるような現象をモデルかするとよく生じる。このような行列を用いた線形方程式を解く場合、反復法の利用を検討するのが良いとされる。

### 反復法の原理

$\mathbf x_{k+1} = B\mathbf x_{k+1} + \mathbf c $から生成されるベクトル列があるベクトル$\alpha$に収束すれば、$\alpha$は$\mathbf x = B\mathbf x + \mathbf c $の解である。

### アルゴリズム

- ヤコビ法
- ガウス・ザイデル法
- SOR法


### ヤコビ法

- A: $n\times n$ matrix
- x: solution n vector
- b: n vector


$A\mathbf x = \mathbf b$

次にAを次のようにdecompositionする：

$$
A = D + R  \  \  \  \ \text{ where } \  \  D = \begin{equation}
\begin{bmatrix}
A_{-n,0}^{-n}  & 0  & 0 & \cdots & \cdots & \cdots & \cdots & 0 \\
0  & A_{-n+1,0}^{-n+1}  & 0  & \ddots & && & \vdots \\
0 & 0  & A_{-n+1,1}^{-n+1} & \cdots  & \ddots & &  & \vdots \\
\vdots & \ddots & \ddots & \ddots & \ddots & \ddots &  & \vdots \\
\vdots & & \ddots & \ddots & \ddots & \ddots & \ddots& \vdots\\
\vdots  & & & \ddots & 0  & A_{n-2,0}^{n-2}  &  0  & 0\\
\vdots  & && & \ddots & 0  & A_{n,-1}^{n-1}&0\\
0 & \cdots &  \cdots & \cdots & \cdots & 0 &0  & A_{n,0}^{n} \\
\end{bmatrix}
\end{equation}, \  \ \text{R は Aの対角要素以外の要素が格納された行列}
$$

### update rule: vector-based

$$
\mathbf x^{(k+1)} = D^{-1}(\mathbf b- R\mathbf x^{(k)})
$$

### update rule: The element-based formula

$$
x_i^{(k+1)} = \frac{1}{a_{ii}}\left(b_i - \sum_{j\neq i}a_{ij}x_j^{(k)}\right)
$$



### Jacobi algorithm

```
[INPUT]
    x0: initial guess to the solution
    A: matrix
    eps: convergence criterion
    max_counter: max number of iteration
    
[OUTPUT] 
    solution when convergence is reached
    
[ALGORITHM]
    n = len(A)
    diag = diag(A) # extract A diag element
    for i = 1 to n:
        diag[i] = 1/diag[i]
    D = n Identity matrix @ diag
    R = A - D
    counter = 0
    
    x_updated = D @ (b - R @ x_0)
    
    while abs(x0 - x_updated) > eps & counter < max_counter:
        x_0 = x_updated
        x_updated = D @ (b - R @ x_0)
        counter += 1
    
    return x_updated
```

### REMARK

- 並列代入法


In [32]:
def jacobi_alg(A, b, x0, eps = 1e-7, max_counter = 5000):
    n = len(A)
    counter = 0
    
    diag = np.diag(A)
    R = A - np.diag(diag)
    diag = np.array([1/i for i in diag])
    
    D = np.diag(diag)
    
    x1 = D @ (b - R @ x0)
    
    #while (max(abs(x0 - x1)) > eps) & (counter < max_counter):
    while ~(np.allclose(x0, x1, atol = eps)) & (counter < max_counter):
        x0 = x1
        x1 = D @ (b - R @ x0)
        counter += 1
        
    return x1

Test

In [33]:
A = np.array([[2., 1.], [5., 7.]], dtype=float)
b = np.array([11., 13.])
x0 =  np.array([1., 1.])

jacobi_alg(A = A, b = b, x0 = x0)

array([ 7.11108566, -3.22216959])

## ガウス・ザイデル法

- 逐次代入法でとくJocibi method

$$
{{x}_{i}}^{\left( k+1 \right)}=\frac{1}{{{a}_{ii}}}\left( {{b}_{i}}-\sum\limits_{j=1}^{i-1}{{{a}_{ij}}{{x}_{j}}^{\left( k+1 \right)}-\sum\limits_{j=i+1}^{n}{{{a}_{ij}}{{x}_{j}}^{\left( k \right)}}} \right)
$$

In [34]:
def seidel(A, x0, b, eps = 1e-7, max_counter = 5000):
    n = len(A)
    d = b.copy()
    counter = 0
    x1 = x0.copy()
    while counter < max_counter:
        for i in range(n):
            tmp = 0
            for j in range(i):
                tmp += A[i,j]*x1[j]
            
            for k in range(i+1, n):
                tmp += A[i, k]*x1[k]
            
            x1[i] = (b[i] - tmp)/A[i, i]
        
        if np.allclose(x0, x1, atol = eps):
            break
        x0 = x1.copy()
        counter += 1
    
    return x1

test

In [35]:
A = np.array([[2., 1.], [5., 7.]], dtype=float)
b = np.array([11., 13.])
x0 =  np.array([1., 1.])
seidel(A = A, b = b, x0 = x0)

array([ 7.11110202, -3.22221573])

### Jacobi methodとgauss-seidel methodの処理時間の比較

In [36]:
%%timeit

A = np.array([[2., 1.], [5., 7.]], dtype=float)
b = np.array([11., 13.])
x0 =  np.array([1., 1.])
seidel(A = A, b = b, x0 = x0)

882 µs ± 37.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [37]:
%%timeit

A = np.array([[2., 1.], [5., 7.]], dtype=float)
b = np.array([11., 13.])
x0 =  np.array([1., 1.])
jacobi_alg(A = A, b = b, x0 = x0)

1.01 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## SOR法

Gasuu-seidel法での本来の修正量に1より大きい加速パラメータ$w$を掛けて、過剰に修正することで、収束をより速くしようとする方法

### update rule

$$
\begin{align}
  &x_i^{m+1}=x_i^m+\delta x_i
\tag{1}\\ 
  &\delta x_i=\frac{\omega}{a_{ii}}
    \left[b_i-\left(\sum^{i-1}_{k=1}a_{ik}x^{m+1}_k+\sum^n_{k=1}a_{ik}x^m_k\right)\right],\ 
    i=1,2,\dots,n,
\tag{2}
\end{align}
$$

### optimal relaxation parameter

- $w \in (1, 2)$, 典型的には1.9が選択される
- ヤコビ法の反復行列のスペクトル半径$\rho$が既知のときは

$$
\begin{equation}
\omega = \frac{2}{1+\sqrt{1-\rho_{jacobi}^2}} \nonumber
\end{equation}
$$

## 幾何学的解釈

<img src = "https://github.com/RyoNakagami/omorikaizuka/blob/master/algorithm/geometric_visual.jpg?raw=true">

<img src = "https://github.com/RyoNakagami/omorikaizuka/blob/master/algorithm/jacobi_alg.jpg?raw=true">

<img src = "https://github.com/RyoNakagami/omorikaizuka/blob/master/algorithm/seidel_alg.jpg?raw=true">

### 反復法の収束の必要十分条件

反復法$\mathbf x_{k+1} = B \mathbf x_{k} + c$が、任意の初期ベクトル$\mathbf x_0$に対して真の解に収束するための必要十分条件は、行列$B$の全ての固有値の絶対値が1より小さいことである。

## 固有値の計算

n次正方行列Aの固有値$\lambda$とそれに対応する固有ベクトル$\mathbf u$は

$$
A\mathbf u = \lambda \mathbf u
$$

を満たす非零ベクトル$\mathbf u$のことである。

固有値の求め方は

$$
\phi(\lambda) = det(A - \lambda I) =0
$$

固有ベクトルの求め方は

$$
(A - \lambda I)\mathbf u = 0
$$

### 固有値を数値計算するポイント

- 行列Aを取り扱いやすい形に相似変換する
- QR法で固有値、固有ベクトルを求める
- 減次: 固有値・固有ベクトルが１組得られたのちに、問題を残りの固有値・固有ベクトルを求める問題に縮小すること
- 原点移動 ：$A$と$A - \mu I$は同一の固有ベクトルを持つことを利用
- 逆反復：より良い近似解を求める手法

### 固有値問題の解きやすさ

- 実対称行列
- 固有値の絶対値が互いに離れている
- 固有値が単純（多重固有値がない）
- 対角化可能行列

### REMARK

- 固有値計算はpackageを使うこと
- `numpy.linalg.eig`


## 演習
### [1]

掃出し方を用いて、次の連立一次方程式を解け

$$
\begin{aligned}
3x - 3z & = 6\\
4x + y & = 4\\
-x + y + 2z &= -3
\end{aligned}
$$

In [38]:
X = np.array(([3, 0, -3, 6] 
             ,[4, 1, 0, 4]
             ,[-1, 1, 2, -3]))
gauss_jordan(X, output_array = True)

array([[ 1,  0,  0,  1],
       [ 0,  1,  0,  0],
       [ 0,  0,  1, -1]])

### [2]


掃出し方を用いて、次の連立一次方程式を解け

$$
\begin{aligned}
-2x + 2y & = 0\\
3x - 3y &+ z = 1\\
2x + y + 6z &= 9
\end{aligned}
$$


In [39]:
X = np.array(([2, 1, 6, 9]
             ,[-2, 2, 0, 0]
             ,[3, -3, 1, 1] 
             ))
gauss_jordan(X, output_array = True)

array([[1, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 1, 1]])

### [3] 行列式とLU分解

$$
A = \left(\begin{array}{ccc}
1 & 0 & 1\\
2 & 1 & 1\\
3 & 4 & 1
\end{array}\right)
$$

をLU分解し、行列式を求めよ

In [40]:
A = np.array([[1, 0, 1]
             ,[2, 1, 1]
             ,[3, 4, 1]], dtype = float)


P, L, U = lu_decomposition(A)
np.linalg.det(P) * np.product(np.diag(U))

2.0